# ReAct Agent From Scratch — Hotel + Weather Teaching Demo

This notebook implements a ReAct-style agent from scratch and walks through the reasoning trace
on a realistic business-travel query (hotel + weather + packing).

In [ ]:
import os
import json
import textwrap
from dataclasses import dataclass
from typing import List, Dict, Callable, Optional

import requests
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL_NAME = "gpt-4o-mini"
print("Using model:", MODEL_NAME)

## Tools: hotel catalog + weather snapshot

In [ ]:
@dataclass
class ToolResult:
    name: str
    input: str
    output: str

def hotel_search_bangalore(query: str) -> str:
    hotels = [
        {
            "name": "Orion Business Hotel",
            "area": "Whitefield",
            "distance_km_to_office": 1.2,
            "price_band_inr": "4500-5500",
            "notes": "Walkable to tech park, good Wi-Fi, quiet, popular with business travelers."
        },
        {
            "name": "Skyline Executive Suites",
            "area": "Indiranagar",
            "distance_km_to_office": 8.0,
            "price_band_inr": "5000-6500",
            "notes": "Great restaurants nearby, slightly longer commute in traffic."
        },
        {
            "name": "Airport Link Business Hotel",
            "area": "Near Airport",
            "distance_km_to_office": 30.0,
            "price_band_inr": "4000-5200",
            "notes": "Good for late-night arrivals, but long commute to office area."
        },
    ]
    lines = ["Bangalore business hotels (internal catalog):", ""]
    for h in hotels:
        lines.append(
            f"- {h['name']} ({h['area']}): "
            f"{h['distance_km_to_office']}km to office, "
            f"{h['price_band_inr']} INR/night. Notes: {h['notes']}"
        )
    lines.append("")
    lines.append("Choose one that balances commute time and comfort based on traveler role and arrival time.")
    return "\n".join(lines)

def weather_snapshot(city: str) -> str:
    try:
        if city.lower().startswith("bangalore") or city.lower().startswith("bengaluru"):
            lat, lon = 12.9716, 77.5946
        else:
            lat, lon = 12.9716, 77.5946

        resp = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={"latitude": lat, "longitude": lon, "current_weather": True},
            timeout=10,
        )
        if resp.status_code != 200:
            return f"[weather] API error {resp.status_code}: {resp.text[:200]}"
        data = resp.json()
        cw = data.get("current_weather", {})
        temp_c = cw.get("temperature")
        wind = cw.get("windspeed")
        cond_code = cw.get("weathercode")
        return (
            f"Current weather for {city}: {temp_c}°C, wind {wind} km/h, "
            f"weathercode {cond_code} (see open-meteo codes). "
            f"Evenings can feel cooler; pack a light jacket."
        )
    except Exception as e:
        return f"[weather] fallback: could not retrieve live data ({e}). Assume warm but variable; pack breathable clothes and a light jacket."

TOOLS: Dict[str, Callable[[str], str]] = {
    "hotel_search_bangalore": hotel_search_bangalore,
    "weather_snapshot": weather_snapshot,
}

TOOL_DESCRIPTIONS = """
You have access to the following tools:

1. hotel_search_bangalore(query: str):
   Use this to get corporate-approved business hotels in Bangalore, including
   distance to the office and price range.

2. weather_snapshot(city: str):
   Use this to get a rough current weather snapshot for a city name.
"""

print(TOOL_DESCRIPTIONS)

## ReAct system prompt and parser

In [ ]:
SYSTEM_PROMPT = f"""
You are a ReAct-style reasoning agent that helps business travelers pick hotels and pack properly.

You must follow this protocol strictly:

1. You reason step by step in "Thought" lines.
2. When you need external information, you use one of the tools below.
3. After calling a tool, you will see an "Observation" with its result.
4. You continue Thought -> Action -> Observation loops as needed.
5. When you are ready, you produce a single "Final Answer".

{TOOL_DESCRIPTIONS}

Format rules:

- If you use a tool, respond with:
  Thought: <your reasoning>
  Action: <one of: hotel_search_bangalore, weather_snapshot>
  Action Input: <a single-line string argument>

- If you want to answer the user, respond with:
  Final Answer: <your complete answer>
"""

def call_llm(messages: List[Dict[str, str]]) -> str:
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.3,
    )
    return response.choices[0].message.content

def parse_react_step(text: str):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for line in lines:
        if line.startswith("Final Answer:"):
            return {"type": "final", "answer": line[len("Final Answer:"):].strip()}
    thought = None
    tool = None
    tool_input = None
    for line in lines:
        if line.startswith("Thought:"):
            thought = line[len("Thought:"):].strip()
        elif line.startswith("Action:"):
            tool = line[len("Action:"):].strip()
        elif line.startswith("Action Input:"):
            tool_input = line[len("Action Input:"):].strip().strip('"')
    if tool and tool_input is not None:
        return {"type": "action", "tool": tool, "tool_input": tool_input, "thought": thought}
    return {"type": "unknown", "raw": text}

## ReAct loop and demo run

In [ ]:
def run_react_agent(user_query: str, max_steps: int = 6):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]
    trace: List[ToolResult] = []
    final_answer = None

    for step in range(1, max_steps + 1):
        print(f"\n=== Step {step} ===")
        assistant_text = call_llm(messages)
        print("LLM raw output:\n", assistant_text)
        parsed = parse_react_step(assistant_text)

        if parsed["type"] == "final":
            final_answer = parsed["answer"]
            print("\n[Agent decided on Final Answer]")
            break

        if parsed["type"] == "action":
            tool_name = parsed["tool"]
            tool_input = parsed["tool_input"]
            print(f"Parsed Action: {tool_name}({tool_input})")
            if tool_name not in TOOLS:
                observation = f"[error] Unknown tool: {tool_name}"
            else:
                observation = TOOLS[tool_name](tool_input)
            trace.append(ToolResult(name=tool_name, input=tool_input, output=observation))
            messages.append({"role": "assistant", "content": assistant_text})
            obs_text = f"Observation: {observation}"
            print("Tool Observation:\n", obs_text)
            messages.append({"role": "user", "content": obs_text})
            continue

        messages.append({"role": "assistant", "content": assistant_text})

    return {"trace": trace, "final_answer": final_answer}

user_query = (
    "I am traveling to Bangalore for a 3-day business trip near the main tech park. "
    "Recommend a good business hotel and tell me what to pack based on the current weather. "
    "I prefer not to spend more than 5500 INR per night."
)

result = run_react_agent(user_query, max_steps=6)
print("\nFINAL ANSWER:\n", result["final_answer"])

## Inspect the tool trace

In [ ]:
for i, tr in enumerate(result["trace"], start=1):
    print(f"\n---------- Tool Call {i} ----------")
    print(f"Tool: {tr.name}")
    print(f"Input: {tr.input}")
    print("Output:")
    print(textwrap.indent(tr.output, prefix="  "))